#🔹 INSTALL REQUIRED LIBRARIES

In [1]:
!pip install streamlit #UI / Web app
!pip install groq       #LLM access,Groq enables access to LLaMA-3 large language models
!pip install PyMuPDF    #PDF text extraction
!pip install fpdf       #PDF download

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 55.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=35b0505afce3cf0fcc6be0f470dbf8b094857e1c89965dfe6dcfb307c6b91e6c
  Stored in directory: /root/.cache/pip/wheels/6e/62/11/dc73d78e40a218ad52e7451f30166e94491be013a7850b5d75
Successfully built fpdf


#🔹 SET API KEY SAFELY

#🔷 STREAMLIT UI

In [3]:
#command: install streamlit and ngrok
!pip install streamlit pyngrok fpdf pymupdf groq


In [9]:
%%writefile app.py
import streamlit as st #UI & app layout
from groq import Groq #AI model access
from fpdf import FPDF #PDF download
import fitz #Read PDF text
import os #Accesses environment variables
import re #Clean text

# ================= PAGE CONFIG =================
st.set_page_config(page_title="Parliament Bill Auditor", layout="wide")
st.title("🏛️ Parliament Bill Auditor")

# ================= GROQ CLIENT =================
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# ================= SESSION STATE =================
st.session_state.setdefault("analysis", "")
st.session_state.setdefault("bill_text", "")
st.session_state.setdefault("chat_history", [])

# ================= PDF EXTRACTION(NLP) =================
def extract_text_from_pdf(pdf_file):
    doc = fitz.open(stream=pdf_file.read(), filetype="pdf")
    return "\n".join(page.get_text() for page in doc)

# ================= TEXT CHUNKING(NLP) =================
def chunk_text(text, size=1500):
    words = text.split()
    chunks, chunk = [], []
    for w in words:
        chunk.append(w)
        if len(" ".join(chunk)) > size:
            chunks.append(" ".join(chunk))
            chunk = []
    if chunk:
        chunks.append(" ".join(chunk))
    return chunks

# ================= BILL ANALYSIS(LLM) =================
def analyze_bill(text):
    chunks = chunk_text(text)[:5] #extract the bill->limit it chuncks->AI->strict analysis format->o/p(structured policy analysis)
    combined = "\n".join(chunks)

    prompt = f"""
You are a public policy analyst.

Analyze the bill and STRICTLY follow this format:

### SECTOR
Mention sector & sub-sector

### OBJECTIVES
- bullet points

### SUMMARY
Detailed explanation in simple English.

### POSITIVE IMPACT
- bullet points

### RISKS & CHALLENGES
- bullet points

### SHORT-TERM IMPACT
- bullet points

### MEDIUM-TERM IMPACT
- bullet points

### LONG-TERM IMPACT
- bullet points

Bill Text:
{combined}
"""
    r = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}], #Uses chat-based format, The entire prompt is treated as a user message
        temperature=0.2 #factual analysis
    )
    return r.choices[0].message.content

# ================= CHAT AI (LLM) =================
def ask_bill_question(bill_text, analysis_text, question):
    prompt = f"""
You are a public policy assistant.

You may answer using:
1. Original bill text
2. AI-generated analysis

If information is unavailable in BOTH, say:
"Not mentioned in the bill or analysis."

--- BILL TEXT ---
{bill_text[:4000]}

--- ANALYSIS ---
{analysis_text[:4000]}

Question:
{question}
"""
    r = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )
    return r.choices[0].message.content

# ================= SECTION PARSER(NLP) =================
def extract_sections(text): #dict
    sections = {
        "sector": "",
        "objectives": "",
        "summary": "",
        "positive": "",
        "risks": "",
        "impact": ""
    }
    current = None
    for line in text.splitlines():
        l = line.lower()
        if "### sector" in l:   #section heading
            current = "sector"  #line belongs to this section
        elif "### objective" in l:
            current = "objectives"
        elif "### summary" in l:
            current = "summary"
        elif "positive impact" in l:
            current = "positive"
        elif "risk" in l:
            current = "risks"
        elif "impact" in l:
            current = "impact"
        elif current:
            sections[current] += line + "\n"
    return sections

# ================= PDF GENERATOR(NLP) =================
def generate_pdf(text):
    pdf = FPDF()    #new pdf
    pdf.add_page()
    pdf.set_font("Arial", size=10)
    safe = re.sub(r"[^\x00-\xFF]", "-", text) #regex -1st latin,2nd latin
    for line in safe.split("\n"):
        pdf.multi_cell(0, 8, line)  #print multi-line text in PDFs,0-full page width,8-line height,wrap
    return pdf.output(dest="S").encode("latin-1") #dest="S" → return PDF as a string

# ================= UI =================
uploaded_file = st.file_uploader("📄 Upload Parliament Bill (PDF only)", type=["pdf"])

if uploaded_file:
    st.success("PDF uploaded successfully")
    st.session_state.bill_text = extract_text_from_pdf(uploaded_file)            #line 22

    if st.button("🟢 GENERATE ANALYSIS"):
        with st.spinner("Analyzing large bill safely..."):
            st.session_state.analysis = analyze_bill(st.session_state.bill_text) #line 40

if st.session_state.analysis:
    sections = extract_sections(st.session_state.analysis)                       #line 112

    tab1, tab2, tab3 = st.tabs(["📊 Sector", "📝 Summary", "⚖️ Impact"])

    with tab1:
        st.subheader("📊 Sector Classification")
        st.write(sections["sector"])

    with tab2:
        st.subheader("🎯 Objectives")
        st.write(sections["objectives"])
        st.subheader("📝 Detailed Summary")
        st.write(sections["summary"])
        pdf = generate_pdf(sections["objectives"] + "\n\n" + sections["summary"]) #line 141
        st.download_button("📥 Download Summary PDF", pdf, "bill_summary.pdf")

    with tab3:
        st.subheader("✅ Positive Impact")
        st.write(sections["positive"])
        st.subheader("⚠️ Risks & Challenges")
        st.write(sections["risks"])
        st.subheader("⏱ Short / Medium / Long Term Impact")
        st.write(sections["impact"])

# ================= CHAT UI =================
if st.session_state.bill_text and st.session_state.analysis:
    st.divider()
    st.subheader("💬 Ask AI about this Bill")

    question = st.text_input(
        "Ask a question about the current bill",
        placeholder="e.g. Explain long-term impact"
    )

    if st.button("Ask AI"):
        if question.strip():
            with st.spinner("Thinking..."):
                answer = ask_bill_question(
                    st.session_state.bill_text,
                    st.session_state.analysis,
                    question
                )                                                                 #line 84
            st.session_state.chat_history.append({"q": question, "a": answer}) #chat_history = conversation memory
        else:
            st.warning("Please enter a question")

    for chat in reversed(st.session_state.chat_history):
        st.markdown(f"**🧑 Question:** {chat['q']}")
        st.markdown(f"**🤖 Answer:** {chat['a']}")
        st.markdown("---")


Overwriting app.py


In [5]:
!pip install pyngrok
from pyngrok import ngrok

In [6]:
# 🧹 Step 1: Clean up old installations and config
!pip uninstall -y pyngrok
!rm -rf /root/.ngrok2
!pip install pyngrok --quiet

# 🧠 Step 2: Import and add your valid authtoken
from pyngrok import ngrok
!ngrok config add-authtoken "34yRMi4zjk8EouYfzRqonOioZtm_3sCnf2k5ZYEVTsKUA7uUk"

# 🚀 Step 3: Run Streamlit + open tunnel correctly
!streamlit run app.py &>/dev/null&
public_url = ngrok.connect(8501)
print(f"✅ Streamlit App Running at: {public_url.public_url}")


Found existing installation: pyngrok 7.5.0
Uninstalling pyngrok-7.5.0:
  Successfully uninstalled pyngrok-7.5.0
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
✅ Streamlit App Running at: https://subpopular-predominant-everette.ngrok-free.dev


In [ ]:
from pyngrok import ngrok

try:
    # Get a list of all active tunnels
    tunnels = ngrok.get_tunnels()

    if tunnels:
        print("Active tunnels:")
        for tunnel in tunnels:
            print(f"- {tunnel.public_url}")
            # Stop each tunnel individually
            ngrok.disconnect(tunnel.public_url)
        print("Attempted to disconnect active ngrok tunnels.")
    else:
        print("No active ngrok tunnels found started by this session.")

except Exception as e:
    print(f"An error occurred while trying to list or disconnect tunnels: {e}")
    print("You may need to manually stop ngrok processes or check your ngrok dashboard.")

Active tunnels:
- https://subpopular-predominant-everette.ngrok-free.dev
Attempted to disconnect active ngrok tunnels.
